In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get the variables from the environment
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

In [2]:
# Connect to PostgreSQL database
try:
    connection = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password,
        port=db_port
    )
    
    # Create a cursor object
    cursor = connection.cursor()

    # Execute SQL queries to get data from all three tables
    query_consumption_gdp = "SELECT * FROM consumption_gdp"  # Query to fetch data from consumption_gdp table
    query_fraction_of_mortality = "SELECT * FROM fraction_of_mortality"  # Query to fetch data from fraction_of_mortality table
    query_per_capita_litres = "SELECT * FROM per_capita_litres"  # Query to fetch data from per_capita_litres table

    # Fetch consumption_gdp data
    cursor.execute(query_consumption_gdp)
    data_consumption_gdp = cursor.fetchall()
    col_names_consumption_gdp = [desc[0] for desc in cursor.description]

    # Fetch fraction_of_mortality data
    cursor.execute(query_fraction_of_mortality)
    data_fraction_of_mortality = cursor.fetchall()
    col_names_fraction_of_mortality = [desc[0] for desc in cursor.description]

    # Fetch per_capita_litres data
    cursor.execute(query_per_capita_litres)
    data_per_capita_litres = cursor.fetchall()
    col_names_per_capita_litres = [desc[0] for desc in cursor.description]

    # Create Pandas DataFrames from the fetched data
    df_consumption_gdp = pd.DataFrame(data_consumption_gdp, columns=col_names_consumption_gdp)
    df_fraction_of_mortality = pd.DataFrame(data_fraction_of_mortality, columns=col_names_fraction_of_mortality)
    df_per_capita_litres = pd.DataFrame(data_per_capita_litres, columns=col_names_per_capita_litres)
    
    # Merge DataFrames on common columns ('entity', 'code', and 'year')
    df_merged_1 = pd.merge(df_consumption_gdp, df_fraction_of_mortality, on=['entity', 'code', 'year'], how='outer')
    df_merged_final = pd.merge(df_merged_1, df_per_capita_litres, on=['entity', 'code', 'year'], how='outer')
    # Display the merged DataFrame
    print(df_merged_final)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


           entity      code  year total_alcohol_consumption_per_capita_x  \
0        Abkhazia  OWID_ABK  2015                                   None   
1     Afghanistan       AFG  2000                                0.00277   
2     Afghanistan       AFG  2002                                   None   
3     Afghanistan       AFG  2003                                   None   
4     Afghanistan       AFG  2004                                   None   
...           ...       ...   ...                                    ...   
6692     Zimbabwe       ZWE  2018                                   None   
6693     Zimbabwe       ZWE  2019                                3.62809   
6694     Zimbabwe       ZWE  2020                                   None   
6695     Zimbabwe       ZWE  2021                                   None   
6696     Zimbabwe       ZWE  2022                                   None   

     gdp_per_capita continent alcohol_attributable_fractions  \
0              None    

In [3]:
# Example: Show the first 5 rows
df_merged_final.head()

,entity,code,year,total_alcohol_consumption_per_capita_x,gdp_per_capita,continent,alcohol_attributable_fractions,total_alcohol_consumption_per_capita_y
0,Abkhazia,OWID_ABK,2015,None,None,Asia,NaN,NaN
1,Afghanistan,AFG,2000,0.00277,None,None,NaN,0.00277
2,Afghanistan,AFG,2002,None,1280.4631,None,NaN,NaN
3,Afghanistan,AFG,2003,None,1292.3335,None,NaN,NaN
4,Afghanistan,AFG,2004,None,1260.0605,None,NaN,NaN
